# Retrieve `OceanDynN` information from LocalizeSL corefiles
`OceanDynN` represents the number of GCMs used to calculate LSL projections for each year-RCP-site

This notebook saves this information in a `.zarr` in a format similar to the projections themselves

In [ ]:
import sys

sys.path.append("../../../")

In [2]:
import pandas as pd
from oct2py import octave
from shared import (
    DIR_SCRATCH,
    DIR_SLR_AR5_IFILES_INT,
    FS,
    LOCALIZESL_COREFILES,
    PATH_LOCALIZESL,
    PATH_SLR_AR5_N_GCMS,
    _to_fuse,
    save,
)

#### Define paths

In [ ]:
DIR_OCTAVE_OUTPUTS = DIR_SCRATCH / "ngcm_localizeSL_outputs"
DIR_OCTAVE_OUTPUTS_FUSE = str(_to_fuse(DIR_OCTAVE_OUTPUTS))

DIR_MFILES = PATH_LOCALIZESL / "MFILES"

### Define Octave function to save information about `OceanDyn` parameters

In [ ]:
load_oceandyn_func = f"""
function oceandyn_info = save_oceandyn_info(corefile_name, subcorefile_choice)
    dir_out = '{DIR_OCTAVE_OUTPUTS_FUSE}';

    ifilesdir='{str(DIR_SLR_AR5_IFILES_INT)}';
    mfilesdir='{str(DIR_MFILES)}';

    addpath(ifilesdir);
    addpath(mfilesdir);

    f = [corefile_name '_v5.mat'];

    corefilewrapper=load(fullfile(ifilesdir, f));

    mkdir(dir_out);

    if strcmp(corefile_name, 'SLRProjections190726core_SEJ_full')
        if strcmp(subcorefile_choice, "H")
            corefile = corefilewrapper.corefileH;
        else
            corefile = corefilewrapper.corefileL;
        end
    else
        corefile = corefilewrapper;
    end

    disp(["Corefile: " corefile_name]);
    disp(["Corefile subgroup: " subcorefile_choice]);

    siteids = int64(corefile.targregions);

    for i=1:length(corefile.scens)
        scen = cell2mat(corefile.scens(i));
        csvwrite(strcat(dir_out, '/OceanDynN_', corefile_name, '_', scen, '.csv'), corefile.OceanDynN(:,:,i));
        csvwrite(strcat(dir_out, '/OceanDynYears_', corefile_name, '_', scen, '.csv'), corefile.OceanDynYears);
        dlmwrite(strcat(dir_out, '/siteid_', corefile_name, '_', scen, '.csv'), siteids, 'precision', '%i')
    end
end
"""

### Save OceanDyn metadata for each corefile

In [ ]:
octave.eval(load_oceandyn_func)

for corefile_name, subcorefiles in LOCALIZESL_COREFILES.items():
    for subcorefile_choice in subcorefiles:
        octave.save_oceandyn_info(corefile_name, subcorefile_choice)

Corefile: SLRProjections190726core_SEJ_full
Corefile subgroup: L
Corefile: SLRProjections190726core_SEJ_full
Corefile subgroup: H
Corefile: SLRProjections170113GRIDDEDcore
Corefile subgroup:  
Corefile: SLRProjections200204GRIDDEDcore_D20
Corefile subgroup:  
Corefile: SLRProjections210628GRIDDEDcore_SROCC
Corefile subgroup:  


### Load CSV outputs into DataFrames

In [ ]:
rcps = ["2p0degree+L", "rcp85+H", "rcp26", "rcp45", "rcp60", "rcp85"]

dfs = []
for corefile in LOCALIZESL_COREFILES.keys():
    rcps = ["rcp26", "rcp45", "rcp60", "rcp85"]
    if corefile == "SLRProjections190726core_SEJ_full":
        rcps = ["2p0degree+L", "rcp85+H"]

    print(corefile, rcps)

    for rcp in rcps:
        scenario = f"{corefile}_{rcp}"
        sites = pd.read_csv(DIR_OCTAVE_OUTPUTS / f"siteid_{scenario}.csv", header=None)
        sites["name"] = sites[0].astype(int).astype(str)
        sites = sites.drop(columns=[0])

        years = pd.read_csv(
            DIR_OCTAVE_OUTPUTS / f"OceanDynYears_{scenario}.csv", header=None
        )

        years = years.T.rename(columns={0: "year"})

        df = pd.read_csv(
            DIR_OCTAVE_OUTPUTS / f"OceanDynN_{scenario}.csv",
            header=None,
            names=sites["name"].values,
        )
        df = df.join(years).set_index("year", drop=True)
        df = df[
            [c for c in df.columns if int(c) > 100000000]
        ]  # These high-valued sites are the gridded ones
        df = df.loc[2000:]
        df["scenario"] = scenario
        df = df.set_index("scenario", append=True)

        df = df.stack()
        df.index = df.index.set_names("scenario", level=1)
        df.index = df.index.set_names("site_id", level=2)

        dfs.append(df)

SLRProjections190726core_SEJ_full ['2p0degree+L', 'rcp85+H']
SLRProjections170113GRIDDEDcore ['rcp26', 'rcp45', 'rcp60', 'rcp85']
SLRProjections200204GRIDDEDcore_D20 ['rcp26', 'rcp45', 'rcp60', 'rcp85']
SLRProjections210628GRIDDEDcore_SROCC ['rcp26', 'rcp45', 'rcp60', 'rcp85']


## Merge DataFrames into xarray and save

In [ ]:
save(pd.concat(dfs).to_xarray().to_dataset(name="numGCMs"), PATH_SLR_AR5_N_GCMS)

In [ ]:
FS.rm(str(DIR_OCTAVE_OUTPUTS), recursive=True)